# Fine-Tuning a Transformer for Multi-Class Classification Using AG News Dataset

# Introduction

This notebook demonstrates the fine-tuning of a pre-trained transformer model for a multi-class text classification task using the AG News dataset. The AG News dataset is widely used in NLP tasks and consists of four distinct categories: World, Sports, Business, and Science/Technology. Each sample in the dataset contains a news article headline and body text, labeled according to its category.

#Objective

The primary goal of this project is to fine-tune the transformer model bert-base-cased for accurate classification of news articles across the four categories. We will utilize the Hugging Face Transformers library to streamline the fine-tuning process and implement a comprehensive evaluation using metrics such as:

Accuracy (Primary Metric)
Precision, Recall, and F1-Score (Macro-averaged)
By following this notebook, you will gain a clear understanding of how to:

Load and explore the AG News dataset.
Preprocess and tokenize the text data using a transformer tokenizer.
Fine-tune a pre-trained transformer model on the training set.
Evaluate the model’s performance on the test set using appropriate metrics for multi-class classification.

In [ ]:
# Transformers installation
! pip install transformers datasets --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.5 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/huggingface/transformers
!pip install /content/transformers

Cloning into 'transformers'...
remote: Enumerating objects: 257190, done.
remote: Counting objects: 100% (30710/30710), done.
remote: Compressing objects: 100% (2371/2371), done.
remote: Total 257190 (delta 29892), reused 28339 (delta 28339), pack-reused 226480 (from 1)
Receiving objects: 100% (257190/257190), 257.97 MiB | 12.18 MiB/s, done.
Resolving deltas: 100% (190684/190684), done.
Processing ./transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.49.0.dev0-py3-none-any.whl size=10770282 sha256=6612ebdd13c2a7be4d3a04767d7310a1814fd84ec283414400219942a0b781d9
  Stored in directory: /tmp/pip-ephem-wheel-cache-sjv5t8q7/wheels/9f/62/72/77fdff469e8308ad837268261590df9cabff9926cc4ab177c0
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-

In [ ]:
!pip install accelerate -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

### ‼️ Restart runtime after installs!

# Fine-tuning the pretrained model on Multi-Class Classification

## Preparing of the dataset

Before we can fine-tune a pretrained model, we download a dataset and prepare it for training.

We begin by loading the [AG News](https://huggingface.co/datasets/sh0416/ag_news) dataset:

In [ ]:
# Importing the dataset and review a row of the dataset.
from datasets import load_dataset

dataset = load_dataset("ag_news")
dataset["train"][10]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

{'text': "Oil and Economy Cloud Stocks' Outlook  NEW YORK (Reuters) - Soaring crude prices plus worries  about the economy and the outlook for earnings are expected to  hang over the stock market next week during the depth of the  summer doldrums.",
 'label': 2}

In [ ]:
# Finding the occurrence of each label
from collections import Counter

label_counts = Counter(dataset['train']['label'])
for label, count in label_counts.items():
    print(f"Label: {label}, Count: {count}")


Label: 2, Count: 30000
Label: 3, Count: 30000
Label: 1, Count: 30000
Label: 0, Count: 30000


In [ ]:
# Creating a smaller subset of the full dataset to fine-tune on to reduce the training time.
small_train_dataset = dataset["train"].shuffle(seed=42).select(range(100))
small_eval_dataset = dataset["test"].shuffle(seed=42).select(range(25))


Now that you know you need a tokenizer to process the text and a padding and truncation strategy to handle different sequence lengths, you can use the [`map`](https://huggingface.co/docs/datasets/process.html#map) method from Hugging Face Datasets to apply a preprocessing function to your entire dataset in one step.



In [ ]:
# Tokenizing the text in the training and evaluation datasets using a BERT tokenizer from HuggingFace.
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(examples):
  return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_small_train_dataset = small_train_dataset.map(tokenize_function, batched=True)
tokenized_small_eval_dataset = small_eval_dataset.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/25 [00:00<?, ? examples/s]

In [ ]:
# Calling for the dimensions of the tokenized training dataset
tokenized_small_train_dataset.shape

(100, 5)

## Training using HuggingFace Transformers Trainer (PyTorch)

In [ ]:
# Loading pre-trained BERT model for the 4-class sequence classification.
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=4)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Training hyperparameters

In [ ]:
# Importing TrainingArguments which are default hyperparameters and save the results to "test_trainer" folder.
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

### Evaluating

In [ ]:
# Installing evaluate library
!pip install evaluate --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.3 MB/s eta 0:00:00


In [ ]:
# Importing numpy and evaluate library and defining the metrics used to evaluate the model
import numpy as np
import evaluate

accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
recall = evaluate.load("recall")

In [ ]:
# Defining a function for the metrics used to evaluate the model (Accuracy, Precision & Recall)
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

   # Compute metrics
    accuracy_score = accuracy.compute(predictions=predictions, references=labels)
    precision_score = precision.compute(predictions=predictions, references=labels, average="macro")
    recall_score = recall.compute(predictions=predictions, references=labels, average="macro")

     # Combine all metrics in a dictionary
    return {
        "accuracy": accuracy_score["accuracy"],
        "precision": precision_score["precision"],
        "recall": recall_score["recall"],
    }


In [ ]:
# Importing TrainingArguments and specifying key parameters like batch size, the number of epochs, -
# where to save outputs and logs, and when to evaluate the model.
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=8,
    eval_strategy="epoch",
    logging_dir='./logs',
)

### Trainer

In [ ]:
# Creating a trainer object including the model, training arguments, training and test datasets and the evaluation function.
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_small_train_dataset,
    eval_dataset=tokenized_small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
# Fine-tuning the model by using .train()
trainer.train()

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall
1,No log,1.144870,0.680000,0.759615,0.647619


TrainOutput(global_step=13, training_loss=1.23077025780311, metrics={'train_runtime': 718.1057, 'train_samples_per_second': 0.139, 'train_steps_per_second': 0.018, 'total_flos': 26311578009600.0, 'train_loss': 1.23077025780311, 'epoch': 1.0})

# Upload to HuggingFace

In [17]:
!pip install huggingface_hub


In [29]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineG

In [27]:
trainer.save_model("fine_tuned_model")
tokenizer.save_pretrained("fine_tuned_model")

from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load the model and tokenizer from the directory where you saved them
model = AutoModelForSequenceClassification.from_pretrained("fine_tuned_model")
tokenizer = AutoTokenizer.from_pretrained("fine_tuned_model")

In [31]:
# Push the model and tokenizer to the Hugging Face Hub
model.push_to_hub("UlrikN123/fine_tuned_model")
tokenizer.push_to_hub("UlrikN123/fine_tuned_model")

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/UlrikN123/fine_tuned_model/commit/52c078fc0d46954ee17147612e4e75f0f5c4cb6c', commit_message='Upload tokenizer', commit_description='', oid='52c078fc0d46954ee17147612e4e75f0f5c4cb6c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/UlrikN123/fine_tuned_model', endpoint='https://huggingface.co', repo_type='model', repo_id='UlrikN123/fine_tuned_model'), pr_revision=None, pr_num=None)